In [1]:
import sys
sys.path.append('..')
from import_library import *

In [2]:
data_type ='var' #mat, csv, mix

# dataloader
clinical_data = pd.read_csv('../data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = '../data/mdALFF matrix excel/' #raw_data_path = './data/image data/dFC matrix matlab/'
if data_type!='mix':
    next_path =f'mdALFF_{data_type}.csv' #next_path ='C0005FC_z.mat't'
else:
    pass
def loader_(data_path = None, data_type='mat' ,mat_dtype='FCM_mean'):
    if '.mat' in data_path:
        mat = scipy.io.loadmat(data_path)
        get_value = mat[mat_dtype ]
        return get_value.shape

    else:
        pd_tb = pd.read_csv(data_path)
        pd_patient_index = pd_tb[pd_tb.columns[0]]
        # pd_patient_value = pd_tb[pd_tb.columns[1::]]
        pd_patient_value = pd_tb
        return pd_patient_index, pd_patient_value
_, patient_value = loader_(data_path = os.path.join(raw_data_path,next_path))
patient_value_index = clinical_data['subj']

In [3]:
for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
    label_ = clinical_data[outcome_name]
    if 'MOCA' in outcome_name:
        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
    elif 'AD8' in outcome_name:
        label_s = [1 if i  >=2 else 0 for i in list(label_)]
    label_ = label_+1
    X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
    y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)
    X_train, X_valid, _, _ = train_test_split(X_train, list(Y_label) , stratify=list(Y_label), test_size=0.5, random_state=123)
    y_train, y_valid, _, _ = train_test_split(y_train, list(Y_label) , stratify=list(Y_label), test_size=0.5,random_state=123)
    X_train['outcome'] = np.array(y_train)
    X_valid['outcome'] = np.array(y_valid)
    X_test['outcome'] = np.array(y_test)
    # concat_df = pd.concat(X_train, )
    X_train['data_type'] = ['train' for i in range(len(X_train))]
    X_valid['data_type'] = ['valid' for i in range(len(X_valid))]
    X_test['data_type'] = ['test' for i in range(len(X_test))]
    df_concat = pd.concat([X_train, X_valid])
    df_concat.to_csv(f'../../R/dataset/{outcome_name}[{data_type}].csv', index=False)